## Sección De Hiperparámetros

In [ ]:
# Hiperparámetros para la carga y preprocesamiento de datos
CLASS_FILTER_AMOUNT = 50           # Número de clases a clasificar
IMAGE_SIZE = 224                   # Tamaño de la imagen de entrada (224x224)
BATCH_SIZE = 64                    # Tamaño del batch para entrenamiento

# Hiperparámetros de entrenamiento
NON_DOWNSAMPLING_KERNEL_SIZE = 3   # Tamaño del kernel para convoluciones sin downsampling
DOWNSAMPLING_KERNEL_SIZE = 1       # Tamaño del kernel para convoluciones con downsampling
NON_DOWNSAMPLING_STRIDE = 1        # Stride para convoluciones sin downsampling
DOWNSAMPLING_STRIDE = 2            # Stride para convoluciones con downsampling
PADDING_SIZE = 1
RESNET20_LAYERS = 3                # Padding para las convoluciones

# Hiperparámetros de regularización y optimización
EARLY_PATIENCE = 13                # Paciencia para early stopping
DROPOUT_RATE_CONV = 0.25           # Tasa de dropout para capas convolucionales (si se utiliza)
DROPOUT_RATE_FC1 = 0.4             # Tasa de dropout para la capa totalmente conectada (ajustada para evitar pérdida excesiva de información)
LEARNING_RATE = 0.001              # Tasa de aprendizaje
NORM_LAYER_INIT_WEIGHT = 1         # Valor de inicialización para pesos de BatchNorm
NORM_LAYER_INIT_BIAS = 0           # Valor de inicialización para bias de BatchNorm
MOMENTUM = 0.9                     # Momentum para el optimizador
EPOCHS = 100                       # Número de épocas de entrenamiento
SCHEDULER_PATIENCE = 6             # Paciencia para el scheduler de tasa de aprendizaje
SCHEDULER_FACTOR = 0.2             # Factor de reducción para el scheduler
WEIGHT_DECAY = 0.0001              # Peso de regularización L2

# Hiperparámetros de seguridad
CHECKPOINT_PATH = './saved_models/plantnet_300k_resnet34_checkpoint.pth' # Ruta para guardar el checkpoint

## Library import and dataset download.

In the next block, the import of the necessary libraries for the operation of the convolutional neural network takes place, together with secondary tools for auxiliary tasks such as graphing, etc. In addition, the set of images to be classified, belonging to a dataset hosted in the Kaggle platform, is downloaded.

In [2]:
import kagglehub
import torch
import os
from torch import tensor
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import ResNet34_Weights, resnet34
import torch.optim as optim
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

# We begin by downloading the required dataset from the kaggle platform.
# The selected dataset is PlantNet-300k by Noah Badoa. It contains approximately 300k images from several plant species.

def download_dataset(dataset_name, version = None, path = None, force_download = False):
    source = f"{dataset_name}/versions/{version}" if version else dataset_name
    return kagglehub.dataset_download(
        source,
        path = path,
        force_download = force_download
    )

dataset_path = download_dataset(dataset_name = "noahbadoa/plantnet-300k-images")

/home/adrian/Dev/Plant-Buddies/backend/model/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Extracting the set of classes with the highest number of images.

Then, according to the number of images in the dataset for each of the classes, we will filter out those with the fifty largest numbers. We aim to facilitate and adapt the model to an adequate classification capacity.

In [3]:
# Specifying the paths to the training, evaluation and test directories.
subpath = 'plantnet_300K'
train_path = os.path.join(dataset_path, f"{subpath}/images_train")
val_path = os.path.join(dataset_path, f"{subpath}/images_val")
test_path = os.path.join(dataset_path, f"{subpath}/images_test")

# We iterate trough the training directory to get the total number of images of each class
class_count = []
for class_id in os.listdir(train_path):
    class_dir = os.path.join(train_path, class_id)
    num_images = len(os.listdir(class_dir))
    class_count.append((class_id, num_images))

# Once we have the number of images of each class, we sort the array and filter the desired ones
class_filter_amount = 50
class_count.sort(key = lambda x: x[1], reverse = True)
top_classes = [cls[0] for cls in class_count[:CLASS_FILTER_AMOUNT]]

# Printing out the ids of the 50 most populated classes.
print(f"\nFifty most populated classes are:\n---------------------------------\n\n{top_classes}")


Fifty most populated classes are:
---------------------------------

['1363227', '1392475', '1356022', '1364099', '1355937', '1359517', '1357330', '1358752', '1359620', '1363128', '1363991', '1355936', '1394460', '1363740', '1394994', '1364173', '1359616', '1364164', '1361824', '1361823', '1397364', '1358095', '1363130', '1389510', '1374048', '1367432', '1409238', '1397268', '1393614', '1356781', '1369887', '1393241', '1394420', '1398178', '1408774', '1435714', '1394591', '1385937', '1355932', '1358094', '1393425', '1393423', '1398592', '1408961', '1358133', '1358766', '1361656', '1384485', '1356257', '1358689']


## Creation of datasets filtered by the obtained classes.

From the test, training and validation directories, several datasets are created, filtering out those images not corresponding to the allowed classes. A python class is implemented by extending ‘ImageFolder’, manipulating its properties as appropriate to obtain the expected results. In addition, the labels associated with each image in a dataset are reassigned so that they belong to a range between zero and the total number of classes minus one. Finally, a primitive transformation is applied to each of the images in the data sets, manipulating the size and shape of the images (tensor transformation).

In [4]:
class PlanetNet300K_Filtered_Dataset(datasets.ImageFolder):
    
    """
    Data classes extending ‘ImageFolder’. Responsible for creating exclusionary datasets, reasigning labels and applying
    unnormalized transformations.

    Attributes
    -----------------------------------
    path (str): path to the folder where the set of images to be filtered is located.
    allowed_classes (list): list with the set of class names to keep. 
    tranform (callable): primitive set of transformations to apply to each filtered image.
    loader (callable): function in charge of loading the set of images from the specified path.
    is_valid_file (callable): function that defines if a found file is valid or not at the time of loading.
    """
    
    def __init__(self, path, allowed_classes, transform=None, loader=datasets.folder.default_loader, is_valid_file=None):
        # The ‘Image Folder’ object is initialized with the received attributes.
        super().__init__(path, transform=transform, loader=loader, is_valid_file=is_valid_file)
        
        # The new mapping is created between the automatically originated tags for each image and the class number to which they belong.
        self.allowed_classes = allowed_classes
        self.new_class_to_idx = {class_name: idx for idx, class_name in enumerate(allowed_classes)}
        
        # The process of filtering and reassigning labels is performed.
        filtered_samples = []
        for path, orig_label in self.samples:
            class_name = self.classes[orig_label]
            if class_name in self.allowed_classes:
                new_label = self.new_class_to_idx[class_name]
                filtered_samples.append((path, new_label))

        # Properties 'samples', 'classes' and 'class_to_idx' are updated.
        self.samples = filtered_samples
        self.classes = allowed_classes
        self.class_to_idx = self.new_class_to_idx

unnormalized_transformation = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor()
])

# Defining training, test and validation datasets
train_dataset = PlanetNet300K_Filtered_Dataset(train_path, top_classes, unnormalized_transformation)
val_dataset = PlanetNet300K_Filtered_Dataset(val_path, top_classes, unnormalized_transformation)
test_dataset = PlanetNet300K_Filtered_Dataset(test_path, top_classes, unnormalized_transformation)

## Computing the global mean and standart desviation for each dataset

The global mean and standard deviation of each of the datasets is calculated, in order to perform a normalization of the pixel values of the images. For this purpose, we use a method that accumulates the sum and sum of squares of all pixels in the dataset.

The overall mean of the data set is calculated according to the following formula:

$mean = \frac{total\,sum\,of\,pixels}{total\,number\,of\,pixels}$

The standard deviation follows the following formula:

$std = \sqrt{\frac{total\,sum\,of\,squares}{total\,number\,of\,pixels} - (mean)^{2}}$

In [ ]:
def compute_mean_std(dataset, batch_size = BATCH_SIZE, num_workers = 0, identifier = None):
    """
    Computes the maan and standard desviation of a given dataset.

    Parameters
    -------------------------------------------------------------
    dataset (ImageFolder): the dataset on which the operations are going to be based
    batch_size (int): number of images from the dataset to be processed in parallel in a single segment.
    num_workers (int): indicates the number of threads (parallel processes) that will be used to load the data.
    """

    loader = DataLoader(dataset,batch_size = batch_size,num_workers = num_workers,pin_memory = True,shuffle = False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Defining variables to keep track of total sum of pixels, total sum of squares and total number of pixels
    dataset_sum_of_pixels = torch.zeros(3, device = device)
    dataset_sum_of_squares = torch.zeros(3, device = device)
    dataset_number_of_pixels = 0

    for batch, _ in loader:
        """
        Each batch (set of images) is represented in the format [b, c, h, w] beign:
        b: number of images
        c: number of channels per image
        h: height of each individual image
        w: width of each individual image
        """
        batch = batch.to(device)
        samples = batch.size(0)
        batch_number_of_pixels = samples * batch.size(2) * batch.size(3)
        dataset_number_of_pixels += batch_number_of_pixels

        # The dimensions of the images are flattened in the form {b, c, h * w}.
        batch = batch.view(samples, batch.size(1), -1)

        # Both sum and sum of squares of all images in the batch are obtained.
        dataset_sum_of_pixels += batch.sum(dim = (0, 2))
        dataset_sum_of_squares += (batch ** 2).sum(dim = (0, 2))

    # Mean and standard deviation of the dataset are calculated and printed out.
    mean = dataset_sum_of_pixels / dataset_number_of_pixels
    std = torch.sqrt((dataset_sum_of_squares / dataset_number_of_pixels) - (mean ** 2))

    print(f"{identifier.capitalize()} dataset statistics\n{'-' * 75}")
    print(f"{identifier.capitalize()} dataset mean: {mean}\n{identifier.capitalize()} dataset std: {std}\n")

    return mean, std

# Definition of mean and std variables for all testing, validation and training datasets
train_mean, train_std = compute_mean_std(train_dataset, BATCH_SIZE, identifier = 'train')
val_mean, val_std = compute_mean_std(val_dataset, BATCH_SIZE, identifier = 'val')
test_mean, test_std = compute_mean_std(test_dataset, BATCH_SIZE, identifier = 'test')

Train dataset statistics
---------------------------------------------------------------------------
Train dataset mean: tensor([0.4399, 0.4692, 0.3228], device='cuda:0')
Train dataset std: tensor([0.2337, 0.2185, 0.2297], device='cuda:0')

Val dataset statistics
---------------------------------------------------------------------------
Val dataset mean: tensor([0.4403, 0.4687, 0.3238], device='cuda:0')
Val dataset std: tensor([0.2345, 0.2186, 0.2302], device='cuda:0')

Test dataset statistics
---------------------------------------------------------------------------
Test dataset mean: tensor([0.4407, 0.4702, 0.3237], device='cuda:0')
Test dataset std: tensor([0.2340, 0.2182, 0.2297], device='cuda:0')



## Creation of data loaders. Data Augmentation.

In the next section, new transformations are defined allowing data augmentation techniques to be performed on the different sets of images. More specifically, on the training dataset, operations
such as angular rotations, horizontal and vertical flips, modifications in contrast, brightness, saturation or other image parameters will be performed. The validation and testing datasets will
be more lightly modified, as they will be used for tasks that do not require these techniques. The previously calculated mean and standard deviation values will be used to normalize the pixels of
all images used in the model. After applying these transformations, data loaders are generated, whose main task is to programmatically introduce the images into the neural network model.

In [6]:
# Defining final transformation for train dataset.
train_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE), antialias = True),
    transforms.RandomRotation(180),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation = 0.05, hue = 0.05),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std)
])

# Defining final transformation for validation dataset.
val_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE), antialias = True),
    transforms.ToTensor(),
    transforms.Normalize(val_mean, val_std)
])

# Defining final tranformation for testing dataset.
test_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE), antialias = True),
    transforms.ToTensor(),
    transforms.Normalize(test_mean, test_std)
])

# Transformations are applied by generating new datasets that manifest these characteristics.
train_dataset = PlanetNet300K_Filtered_Dataset(train_path, top_classes, train_transforms)
test_dataset = PlanetNet300K_Filtered_Dataset(test_path, top_classes, test_transforms)
val_dataset = PlanetNet300K_Filtered_Dataset(val_path, top_classes, val_transforms)

# Defining data loaders for training, testing and validation
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 0, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 0, pin_memory = True)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 0, pin_memory = True)

## Early Stopping

The early stopping mechanism for model training is established. This technique helps to avoid overfitting and save training time. To detect whether the model should be stopped, the loss produced in the
validation stage is used, which if it does not decrease in a certain number of stages produces the stopping mechanism.

In [8]:
class Early_Stopper:
    """
    Class in charge of stopping the training process if the validation loss does not improve after a certain number of epochs.

    Attributes
    -----------------------------------
    patience (int): number of epochs to wait before stopping the training process.
    counter (int): number of epochs without improvement.
    best_loss (float): best loss obtained during the training process.
    stop (bool): flag that indicates if the training process should be stopped.
    """
    
    def __init__(self, patience = EARLY_PATIENCE, callback = lambda **_: None):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.stop = False
        self.callback = callback

    
    @property
    def counter(self):
        return self._counter
    
    @counter.setter
    def counter(self, value):
        self._counter = value

    def __call__(self, val_loss):
        """
        The method is called to update the counter and the best loss obtained during the training process.

        Parameters
        -----------------------------------
        val_loss (float): loss obtained during the validation process.
        """
        
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.callback()
                self.stop = True

        return self.stop

## Model architecture. Layers and distribution.

Convolutional network model characterized by having and using two types of structurally and functionally differentiated layers. First, we find the convolution layers, so called because of the
mathematical operation they perform on the image pixels. They are mainly in charge of extracting the relevant features or patterns in the images in order to classify them later on. Secondly,
we find the classification layers, whose function is to classify the features resulting from applying the different convolution layers. They are responsible for defining to which class each of
the images that pass through the model belongs. Finally, a series of secondary but very important layers are used, including pooling layers, normalization layers and dropout layers, each with a
role specified in the architecture.

In [9]:
# Definition of a restnet34 model
class ResNet34(nn.Module):
    def __init__(self):
        super().__init__()
        """
        ResNet34 model with a custom classifier. The model is pretrained with the ImageNet dataset.
        First convolutional layer is described with 3 input channels, 64 output channels, kernel size of 3, stride of 1 and padding of 1.
        First fully connected layer is described with ResNet34's number of features as input, 512 output features.
        Second fully connected layer is described with 512 input features and number of classes as output features.
        """
        self.resnet34 = resnet34(weights = ResNet34_Weights.IMAGENET1K_V1)
        num_features = self.resnet34.fc.in_features
        self.resnet34.conv1 = nn.Conv2d(3, 64, kernel_size = NON_DOWNSAMPLING_KERNEL_SIZE, stride = NON_DOWNSAMPLING_STRIDE, padding = PADDING_SIZE, bias = False)
        self.resnet34.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(DROPOUT_RATE_FC1),
            nn.Linear(512, CLASS_FILTER_AMOUNT)
        )

    def forward(self, x):
        return self.resnet34(x)


## Evaluation Section

Function where the evaluation process of the parametric settings of the convolutional neural network will be carried out during the training procedure. The values collected in each evaluation process correspond to the comparison between the preditions and the real values, the reliability of the model and the accuracy of the predictions on failures and hits.

In [10]:
def eval(model, loader, criterion, device):
    """
    Evaluates the performance of the model on a given dataset.

    Parameters
    -------------------------------------------------------------
    model (nn.Module): model to be evaluated.
    loader (DataLoader): data loader containing the dataset to be evaluated.
    criterion (callable): loss function to be used.
    device (torch.device): device on which the operations are going to be performed.
    """ 
    
    if not (device and criterion): raise Exception("Device and criterion must be specified on eval function.")
    
    model.eval()
    accurate_predictions = 0
    total_loss = 0
    total_samples = 0
    predictions = []
    targets = []

    with torch.no_grad():
        for batch, target in loader:
            batch, target = batch.to(device), target.to(device)
            output = model(batch)
            total_loss += criterion(output, target).item()
            _, predicted = torch.max(output, 1)
            accurate_predictions += (predicted == target).sum().item()
            total_samples += target.size(0)
            predictions.extend(predicted.tolist())
            targets.extend(target.tolist())

    return predictions, targets, 100 * accurate_predictions / total_samples, total_loss / len(loader)

# Save and load checkpoint functions.

We define two functions, allowing us to store the weights and statistics provided by the neural network during the training process, so that, in case of an accident, we can restore the process from the last stored record.

In [11]:
# Definition of chechpoint save function
def save_checkpoint(model, scheduler, optimizer, epoch, tracking_lists, early_patience, path = CHECKPOINT_PATH):
    """
    Saves the current state of the model, optimizer and scheduler to a file.

    Parameters
    -------------------------------------------------------------
    model (nn.Module): model to be saved.
    scheduler (callable): scheduler to be saved.
    optimizer (callable): optimizer to be saved.
    epoch (int): current epoch.
    tracking_lists (list): list of tracking values to be saved.
    early_patience (int): current patience for early stopping.
    path (str): path to the file where the model is going to be saved.
    """
    
    torch.save({
        'model': model.state_dict(),
        'scheduler': scheduler.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
        'tracking': tracking_lists,
        'early_patience': early_patience
    }, path)

# Definition of checkpoint load function
def load_checkpoint(path):
    """
    Loads the state of the model, optimizer and scheduler from a file.

    Parameters
    -------------------------------------------------------------
    path (str): path to the file where the model is going to be loaded.
    """
    
    checkpoint = torch.load(path)
    return checkpoint
    

## Training Loop

Function that executes the training loop upon the neural network. The model is successively trained through a series of stages or epochs. In each of them, the parameters governing the performance of the network are reassigned according to the loss detected during the process. In addition, an evaluation of each of the iterative adjustments made to the parameters guiding the network is performed. Finally, data concerning the efficiency of the model after the training process are collected, including the hit and miss accuracies in each of the stages performed.

In [13]:
def train(model, loaders, optimizer, criterion, scheduler, tracking_lists, callback, early_stopper, start_epoch, epochs = EPOCHS, device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    """
    Trains the model on a given dataset.

    Parameters
    -------------------------------------------------------------
    model (nn.Module): model to be trained.
    loaders (dict): dictionary containing the training and validation data loaders.
    optimizer (callable): optimization algorithm to be used.
    criterion (callable): loss function to be used.
    epochs (int): number of epochs to train the model.
    scheduler (callable): learning rate scheduler.
    device (torch.device): device on which the operations are going to be performed.
    early_stopper (callable): object in charge of stopping the training process if the validation loss does not improve.
    callback (callable): function to be called during training loop to print statistics on display.
    """

    if not (optimizer and criterion and early_stopper and scheduler and callback): raise Exception("Optimizer, criterion, early_stopper, scheduler and callback must be specified on train function.")
    model.to(device)

    # Evaluating the model on the validation dataset before starting the training process
    if start_epoch == 0:
        val_predictions, val_targets, val_accuracy, val_loss = eval(model, loaders['val'], criterion, device)
        callback(epoch = 0, val_accuracy = val_accuracy, val_loss = val_loss, train_accuracy = 0, train_loss = float('inf'))
    
    # Starting the training process
    for epoch in range(start_epoch, epochs):
        total_loss = 0
        accurate_predictions = 0
        total_samples = 0
        model.train()

        # Iterating through the training dataset
        for i, (batch, target) in enumerate(loaders['train'], 1):
            batch, target = batch.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(batch)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            accurate_predictions += (predicted == target).sum().item()
            total_samples += target.size(0)
            if not i % 100:
                print(f"\tEpoch {epoch + 1}, Batch {i}, Loss: {loss.item()}")

        # Evaluating the model for the current epoch 
        val_predictions, val_targets, val_accuracy, val_loss = eval(model, loaders['val'], criterion, device)
        scheduler.step(val_loss)

        # Analyzing the performance of the model on the current epoch
        train_accuracy = 100 * accurate_predictions / total_samples
        train_loss = total_loss / len(loaders['train'])
        tracking_lists['train_accuracies'].append(train_accuracy)
        tracking_lists['train_losses'].append(train_loss)
        tracking_lists['eval_accuracies'].append(val_accuracy)
        tracking_lists['eval_losses'].append(val_loss)

        # Printing out the statistics of the model on the current epoch
        callback(epoch = epoch + 1, val_accuracy = val_accuracy, val_loss = val_loss, train_accuracy = train_accuracy, train_loss = train_loss)

        # Saving the model periodically
        if not (epoch + 1) % 5:
            save_checkpoint(model, scheduler, optimizer, epoch + 1, tracking_lists, early_stopper.counter, CHECKPOINT_PATH)

        # Checking if the training process should be stopped
        if early_stopper(val_loss): break

    return tracking_lists, val_predictions, val_targets

## Final Configuration And Training Launch

The final step is to configure the various parameters to be used during training, in the form of optimizations, selected loss function, scheduler, learning rate, etc. The callback functions that will be used to print the model analytics during learning are defined. The model is executed.

In [15]:
def early_callback():
    """
    Function in charge of printing out a message when the training process is stopped by the early stopper.
    """
    print("\nEarly stopping criterion met. Stopping training process.\n")

def callback(epoch, val_accuracy, val_loss, train_accuracy, train_loss):
    """
    Function in charge of printing out the statistics of the model on the current epoch.

    Parameters
    -------------------------------------------------------------
    epoch (int): current epoch.
    val_accuracy (float): accuracy of the model on the validation dataset.
    val_loss (float): loss of the model on the validation dataset.
    train_accuracy (float): accuracy of the model on the training dataset.
    train_loss (float): loss of the model on the training dataset.
    """
    print(f"Epoch: {epoch}, Train Accuracy: {train_accuracy:.2f}%, Train Loss: {train_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, Val Loss: {val_loss:.4f}")

def execute():
    """
    Function in charge of executing the training process of the convolutional neural network.
    """

    # Defining the model, the optimizer, the loss function, the learning rate scheduler and the early stopper
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ResNet34()
    optimizer = optim.SGD(model.parameters(), lr = LEARNING_RATE, momentum = MOMENTUM, nesterov = True, weight_decay = WEIGHT_DECAY)
    
    criterion = nn.CrossEntropyLoss().to(device)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = SCHEDULER_PATIENCE, factor = SCHEDULER_FACTOR, verbose = True)
    early_stopper = Early_Stopper(callback = early_callback)

    # Defining lists to keep track of the training process
    train_accuracies_history = []
    train_losses_history = []
    eval_accuracies_history = []
    eval_losses_history = []

    # Checking if a checkpoint is available
    if os.path.exists(CHECKPOINT_PATH):
        checkpoint = load_checkpoint(CHECKPOINT_PATH)
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        start_epoch = checkpoint['epoch']
        tracking_lists = checkpoint['tracking']
        early_stopper.counter = checkpoint['early_patience']

        # Mover los tensores del estado del optimizador al dispositivo cuda
        for state in optimizer.state.values():
            for key, value in state.items():
                if isinstance(value, torch.Tensor):
                    state[key] = value.to(device)
                    
    else:
        start_epoch = 0
        tracking_lists = {
            'train_accuracies': train_accuracies_history,
            'train_losses': train_losses_history,
            'eval_accuracies': eval_accuracies_history,
            'eval_losses': eval_losses_history
        }

    # Defining the data loaders
    loaders = {
        'train': train_loader,
        'val': val_loader
    }

    print(start_epoch)

    # Training the model
    postrained_tracking_lists, val_predictions, val_targets = train(model, loaders, optimizer, criterion, scheduler, tracking_lists, callback, early_stopper, start_epoch, epochs = EPOCHS, device = device)

execute()

/home/adrian/Dev/Plant-Buddies/backend/model/.venv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


20


KeyboardInterrupt: 